In [1]:
import openreview
from deepdiff import DeepDiff
from pprint import pprint
from __future__ import print_function

guest_client = openreview.Client(baseurl="https://api.openreview.net")

def sort_by_tcdate(notes):
    if type(notes[0]) is dict:
        return list(sorted(notes, key=lambda x: x['tcdate']))
    else:
        return list(sorted(notes, key=lambda x: x.tcdate))

def print_metadata(note):
    if type(note) is dict:
        print(
            note['id'], note['forum'], 
            note['original'] if 'original' in note else None, 
            note['referent'] if 'referent' in note else None, 
            note['cdate'], note['tcdate'], note['tmdate'], note['invitation']
        )
    else:
        print(
            note.id, note.forum, note.original, note.referent,
            note.cdate, note.tcdate, note.tmdate, note.invitation
        )

In [2]:
VENUE = "ICLR.cc/2020/Conference"

In [3]:
# get all blind submissions
forum_notes = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2213


### Referent submission and comment notes

In [4]:
# get a blind submission
forum_note = forum_notes[0]
print('id forum original referent cdate tcdate tmdate invitation')
print_metadata(forum_note)
print(forum_note)

id forum original referent cdate tcdate tmdate invitation
ryxyCeHtPB ryxyCeHtPB SkltbEzFwS None 1569439942705 1569439942705 1644493566951 ICLR.cc/2020/Conference/-/Blind_Submission
{'cdate': 1569439942705,
 'content': {'TL;DR': 'We introduce attentive feature distillation and '
                      'selection, to fine-tune a large model and produce a '
                      'faster one.',
             '_bibtex': '@inproceedings{\n'
                        'Wang2020Pay,\n'
                        'title={Pay Attention to Features, Transfer Learn '
                        'Faster CNNs},\n'
                        'author={Kafeng Wang and Xitong Gao and Yiren Zhao and '
                        'Xingjian Li and Dejing Dou and Cheng-Zhong Xu},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                        'url={https://openreview.net/forum?id=ryxyCeHtPB}\n'
        

In [5]:
# get the referent submission (forum) note and comment notes for this blind submission
this_forum_notes = sort_by_tcdate(list(openreview.tools.iterget_notes(guest_client, forum=forum_note.id)))
print('#NotesThisBlindSubmissionForum', len(this_forum_notes))

print('id forum original referent cdate tcdate tmdate invitation')
for note in this_forum_notes:
    print_metadata(note)
    if note.id == forum_note.id:
        print(DeepDiff(note, forum_note, ignore_order=True))
    else:
        comment_note = note
assert forum_note.id in [note.id for note in this_forum_notes]

#NotesThisBlindSubmissionForum 9
id forum original referent cdate tcdate tmdate invitation
ryxyCeHtPB ryxyCeHtPB SkltbEzFwS None 1569439942705 1569439942705 1644493566951 ICLR.cc/2020/Conference/-/Blind_Submission
{}
Skg8EJfGuH ryxyCeHtPB None None 1570017070423 1570017070423 1570017080129 ICLR.cc/2020/Conference/Paper2594/-/Public_Comment
Ske1UQAX9B ryxyCeHtPB None None 1572229959099 1572229959099 1572972317734 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
BkeROrlU5B ryxyCeHtPB None None 1572369782324 1572369782324 1572972317688 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
BklJwjhd5B ryxyCeHtPB None None 1572551510593 1572551510593 1572972317644 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
BJlAoqSIsH ryxyCeHtPB None None 1573440166129 1573440166129 1573440166129 ICLR.cc/2020/Conference/Paper2594/-/Official_Comment
BJxy8nSUiS ryxyCeHtPB None None 1573440582977 1573440582977 1573440867870 ICLR.cc/2020/Conference/Paper2594/-/Official_Comment
r1e9TnrIsH ryxyCeHtPB None

### Referent submission and its references

In [6]:
print("forum id\t", forum_note.id)
print("original id\t", forum_note.original)

forum id	 ryxyCeHtPB
original id	 SkltbEzFwS


In [7]:
# original submission and its revisions
submission_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=True))
print('#References:', len(submission_references_oT))

has_original = False
print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oT:
    assert ref.original == forum_note.original
    print_metadata(ref)
    if ref.id == forum_note.original:
        print("above is the original submission note")
        has_original = True
    # TODO check that the original has the same contents as the forum
    # TODO only need one of this and the forum note
if not has_original:
    print("MISSING THE ORIGINAL NOTE")

#References: 9
id forum original referent cdate tcdate tmdate invitation
HytZVfFwB ryxyCeHtPB SkltbEzFwS ryxyCeHtPB 1569439942705 1569428480660 1569428480660 ICLR.cc/2020/Conference/-/Blind_Submission
d5vH8qGhI ryxyCeHtPB SkltbEzFwS ryxyCeHtPB 1569439942705 1581671983249 1581671983249 ICLR.cc/2020/Conference/-/Blind_Submission
75Aw9e99A ryxyCeHtPB SkltbEzFwS ryxyCeHtPB 1569439942705 1581672025553 1581672025553 ICLR.cc/2020/Conference/-/Blind_Submission
n9nmLgKMi ryxyCeHtPB SkltbEzFwS ryxyCeHtPB 1569439942705 1581675299026 1581675299026 ICLR.cc/2020/Conference/-/Blind_Submission
uEgZNRmqa ryxyCeHtPB SkltbEzFwS ryxyCeHtPB 1569439942705 1581682811359 1581682811359 ICLR.cc/2020/Conference/-/Blind_Submission
AMZ956OTf ryxyCeHtPB SkltbEzFwS ryxyCeHtPB 1569439942705 1581683094010 1581683094010 ICLR.cc/2020/Conference/-/Blind_Submission
O7mAzvwB9 ryxyCeHtPB SkltbEzFwS ryxyCeHtPB 1569439942705 1581697868549 1581697868549 ICLR.cc/2020/Conference/-/Blind_Submission
8ctugnxRc ryxyCeHtPB SkltbEzFwS

In [8]:
# this seems to return one of the above revisions, which is Code
guest_client.get_all_references(referent=forum_note.original, original=True)

[Note(id = 'rrxvZsuMJc',original = None,number = 220,cdate = 1644493566951,mdate = 1644493566951,tcdate = 1644493566951,tmdate = 1644493566951,ddate = None,content = {'data': '[Caltech-256](https://paperswithcode.com/dataset/caltech-256), [Food-101](https://paperswithcode.com/dataset/food-101)'},forum = 'SkltbEzFwS',referent = 'SkltbEzFwS',invitation = 'PapersWithCode.com/-/ICLR_2020_Code',replyto = None,readers = ['everyone'],nonreaders = [],signatures = ['PapersWithCode.com'],writers = [],details = {})]

In [9]:
guest_client.get_all_references(referent=forum_note.original, original=False)

[Note(id = 'rrxvZsuMJc',original = None,number = 220,cdate = 1644493566951,mdate = 1644493566951,tcdate = 1644493566951,tmdate = 1644493566951,ddate = None,content = {'data': '[Caltech-256](https://paperswithcode.com/dataset/caltech-256), [Food-101](https://paperswithcode.com/dataset/food-101)'},forum = 'SkltbEzFwS',referent = 'SkltbEzFwS',invitation = 'PapersWithCode.com/-/ICLR_2020_Code',replyto = None,readers = ['everyone'],nonreaders = [],signatures = ['PapersWithCode.com'],writers = [],details = None)]

In [10]:
print(forum_note)

{'cdate': 1569439942705,
 'content': {'TL;DR': 'We introduce attentive feature distillation and '
                      'selection, to fine-tune a large model and produce a '
                      'faster one.',
             '_bibtex': '@inproceedings{\n'
                        'Wang2020Pay,\n'
                        'title={Pay Attention to Features, Transfer Learn '
                        'Faster CNNs},\n'
                        'author={Kafeng Wang and Xitong Gao and Yiren Zhao and '
                        'Xingjian Li and Dejing Dou and Cheng-Zhong Xu},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                        'url={https://openreview.net/forum?id=ryxyCeHtPB}\n'
                        '}',
             'abstract': 'Deep convolutional neural networks are now widely '
                         'deployed in vision applications, but a limited size '
  

In [11]:
print(submission_references_oT[0]) # original

{'cdate': 1569439942705,
 'content': {'TL;DR': 'We introduce attentive feature distillation and '
                      'selection, to fine-tune a large model and produce a '
                      'faster one.',
             '_bibtex': '@inproceedings{\n'
                        'Wang2020Pay,\n'
                        'title={Pay Attention to Features, Transfer Learn '
                        'Faster CNNs},\n'
                        'author={Kafeng Wang and Xitong Gao and Yiren Zhao and '
                        'Xingjian Li and Dejing Dou and Cheng-Zhong Xu},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                        'url={https://openreview.net/forum?id=ryxyCeHtPB}\n'
                        '}',
             'abstract': 'Deep convolutional neural networks are now widely '
                         'deployed in vision applications, but the size of '
     

In [12]:
print(submission_references_oT[1]) # revision

{'cdate': 1569439942705,
 'content': {'TL;DR': 'We introduce attentive feature distillation and '
                      'selection, to fine-tune a large model and produce a '
                      'faster one.',
             '_bibtex': '@inproceedings{\n'
                        'Wang2020Pay,\n'
                        'title={Pay Attention to Features, Transfer Learn '
                        'Faster CNNs},\n'
                        'author={Kafeng Wang and Xitong Gao and Yiren Zhao and '
                        'Xingjian Li and Dejing Dou and Cheng-Zhong Xu},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                        'url={https://openreview.net/forum?id=ryxyCeHtPB}\n'
                        '}',
             'abstract': 'Deep convolutional neural networks\n'
                         'are now widely deployed\n'
                         'in vision applica

### A referent comment note and its references

In [13]:
comment_references_oT = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=True))
print('#References:', len(comment_references_oT))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oT:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
5Q38bBkdtq ryxyCeHtPB None EE4Ml5hZtI 1576798753000 1576798753000 1576800882379 ICLR.cc/2020/Conference/Paper2594/-/Decision


In [14]:
comment_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=comment_note.id, original=False))
print('#References:', len(comment_references_oF))
print('id forum original referent cdate tcdate tmdate invitation')
for ref in comment_references_oF:
    print_metadata(ref)

#References: 1
id forum original referent cdate tcdate tmdate invitation
5Q38bBkdtq ryxyCeHtPB None EE4Ml5hZtI 1576798753000 1576798753000 1576800882379 ICLR.cc/2020/Conference/Paper2594/-/Decision


In [15]:
print(comment_note)

{'cdate': 1576798753000,
 'content': {'comment': 'This paper presents an attention-based approach to '
                        'transfer faster CNNs, which tackles the problem of '
                        'jointly transferring source knowledge and pruning '
                        'target CNNs.\n'
                        '\n'
                        'Reviewers are unanimously positive on the paper, in '
                        'terms of a well-written paper with a reasonable '
                        'approach that yields strong empirical performance '
                        'under the resource constraint.\n'
                        '\n'
                        'AC feels that the paper studies an important problem '
                        'of making transfer learning faster for CNNs, however, '
                        'the proposed model is a relatively straightforward '
                        'combination of fine-tuning and filter-pruning, each '
                        'having ver

In [16]:
print(comment_references_oT[0])

{'cdate': 1576798753000,
 'content': {'comment': 'This paper presents an attention-based approach to '
                        'transfer faster CNNs, which tackles the problem of '
                        'jointly transferring source knowledge and pruning '
                        'target CNNs.\n'
                        '\n'
                        'Reviewers are unanimously positive on the paper, in '
                        'terms of a well-written paper with a reasonable '
                        'approach that yields strong empirical performance '
                        'under the resource constraint.\n'
                        '\n'
                        'AC feels that the paper studies an important problem '
                        'of making transfer learning faster for CNNs, however, '
                        'the proposed model is a relatively straightforward '
                        'combination of fine-tuning and filter-pruning, each '
                        'having ver

In [17]:
print(comment_references_oF[0])

{'cdate': 1576798753000,
 'content': {'comment': 'This paper presents an attention-based approach to '
                        'transfer faster CNNs, which tackles the problem of '
                        'jointly transferring source knowledge and pruning '
                        'target CNNs.\n'
                        '\n'
                        'Reviewers are unanimously positive on the paper, in '
                        'terms of a well-written paper with a reasonable '
                        'approach that yields strong empirical performance '
                        'under the resource constraint.\n'
                        '\n'
                        'AC feels that the paper studies an important problem '
                        'of making transfer learning faster for CNNs, however, '
                        'the proposed model is a relatively straightforward '
                        'combination of fine-tuning and filter-pruning, each '
                        'having ver

## Other exploration

### Get notes with direct or all replies

In [18]:
# get all blind submissions, each with its direct replies
forum_notes_dr = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='directReplies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2213


In [19]:
# get all blind submissions, each with its replies
forum_notes_r = list(openreview.tools.iterget_notes(
    guest_client, invitation=f"{VENUE}/-/Blind_Submission", sort='tcdate',
    details='replies'
))
for forum_note in forum_notes:
    assert forum_note.id == forum_note.forum and forum_note.id != forum_note.original
print('#BlindSubmission:', len(forum_notes))

#BlindSubmission: 2213


In [20]:
# the first blind submission with direct replies
forum_note = forum_notes_dr[0]
print('#directReplies:', len(forum_note.details['directReplies']))
for reply in sort_by_tcdate(forum_note.details['directReplies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#directReplies: 5
Skg8EJfGuH ryxyCeHtPB None None 1570017070423 1570017070423 1570017080129 ICLR.cc/2020/Conference/Paper2594/-/Public_Comment
Ske1UQAX9B ryxyCeHtPB None None 1572229959099 1572229959099 1572972317734 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
BkeROrlU5B ryxyCeHtPB None None 1572369782324 1572369782324 1572972317688 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
BklJwjhd5B ryxyCeHtPB None None 1572551510593 1572551510593 1572972317644 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
EE4Ml5hZtI ryxyCeHtPB None None 1576798753000 1576798753000 1576800882379 ICLR.cc/2020/Conference/Paper2594/-/Decision

ryxyCeHtPB ryxyCeHtPB SkltbEzFwS None 1569439942705 1569439942705 1644493566951 ICLR.cc/2020/Conference/-/Blind_Submission
{'cdate': 1569439942705,
 'content': {'TL;DR': 'We introduce attentive feature distillation and '
                      'selection, to fine-tune a large model and produce a '
                      'faster one.',
             '_bibtex': 

In [21]:
# the first blind submission with all replies
forum_note = forum_notes_r[0]
print('#replies:', len(forum_note.details['replies']))
for reply in sort_by_tcdate(forum_note.details['replies']):
    print_metadata(reply)
print()
print_metadata(forum_note)
print(forum_note)

#replies: 8
Skg8EJfGuH ryxyCeHtPB None None 1570017070423 1570017070423 1570017080129 ICLR.cc/2020/Conference/Paper2594/-/Public_Comment
Ske1UQAX9B ryxyCeHtPB None None 1572229959099 1572229959099 1572972317734 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
BkeROrlU5B ryxyCeHtPB None None 1572369782324 1572369782324 1572972317688 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
BklJwjhd5B ryxyCeHtPB None None 1572551510593 1572551510593 1572972317644 ICLR.cc/2020/Conference/Paper2594/-/Official_Review
BJlAoqSIsH ryxyCeHtPB None None 1573440166129 1573440166129 1573440166129 ICLR.cc/2020/Conference/Paper2594/-/Official_Comment
BJxy8nSUiS ryxyCeHtPB None None 1573440582977 1573440582977 1573440867870 ICLR.cc/2020/Conference/Paper2594/-/Official_Comment
r1e9TnrIsH ryxyCeHtPB None None 1573440706488 1573440706488 1573440706488 ICLR.cc/2020/Conference/Paper2594/-/Official_Comment
EE4Ml5hZtI ryxyCeHtPB None None 1576798753000 1576798753000 1576800882379 ICLR.cc/2020/Conference/Paper2

### Get references with other settings

In [22]:
# referent=forum_note.id, original=False
# -> for ICLR 2018 this returns the submission/forum note with little content
# -> here it returns some new note with little content
submission_references_oF = sort_by_tcdate(guest_client.get_all_references(referent=forum_note.id, original=False))
print('#References:', len(submission_references_oF))
assert len(submission_references_oF) == 1

print('id forum original referent cdate tcdate tmdate invitation')
for ref in submission_references_oF:
    print_metadata(ref)
    # assert ref.id == forum_note.id
    assert ref.original == forum_note.original

#References: 1
id forum original referent cdate tcdate tmdate invitation
B1JClBKPH ryxyCeHtPB SkltbEzFwS ryxyCeHtPB 1569439942705 1569439942705 1583912019895 ICLR.cc/2020/Conference/-/Blind_Submission


In [23]:
print(submission_references_oF[0])

{'cdate': 1569439942705,
 'content': {'_bibtex': '@inproceedings{\n'
                        'Wang2020Pay,\n'
                        'title={Pay Attention to Features, Transfer Learn '
                        'Faster CNNs},\n'
                        'author={Kafeng Wang and Xitong Gao and Yiren Zhao and '
                        'Xingjian Li and Dejing Dou and Cheng-Zhong Xu},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                        'url={https://openreview.net/forum?id=ryxyCeHtPB}\n'
                        '}',
             'appendix': '',
             'authorids': ['kf.wang@siat.ac.cn',
                           'xt.gao@siat.ac.cn',
                           'yiren.zhao@cl.cam.ac.uk',
                           'lixingjian@baidu.com',
                           'doudejing@baidu.com',
                           'czxu@um.edu.mo'],
             'author

### Seemingly useful but actually confusing

In [24]:
# get venue-specific event types
# the problem is that, unexpectedly, this doesn't return Official_Review
invitations = openreview.tools.iterget_invitations(
        guest_client, regex=f'{VENUE}.*'
)
event_types = set()
for invitation in invitations:
    event_types.add(invitation.id.split('/')[-1])
print(event_types)

{'Official_Comment', 'Blind_Submission', 'Desk_Rejected_Submission', 'Public_Comment', 'Supplementary_Material', 'Comment', 'Revision', 'Registration', 'Withdrawn_Submission'}
